# Using the fully connected layers

calculate the distance of the vectors extracted by the fully connected layers from VGG and RESNET

In [1]:
import numpy as np
import torchvision
import glob
import pandas as pd
import os

from PIL import Image
from typing import Callable
from scipy.spatial import distance

#### Import the three models, register hooks on their first fully connected layers

In [2]:
# import the models
vgg16 = torchvision.models.vgg16(pretrained=True)
vgg19 = torchvision.models.vgg19(pretrained=True)
alexnet = torchvision.models.alexnet(pretrained=True)

features = {}
def reg_hook(layer: int) -> Callable:
    def hook(model, input, output):
        features[layer] = output.detach()
    return hook
    

vgg16.classifier[0].register_forward_hook(reg_hook("feats"))
vgg19.classifier[0].register_forward_hook(reg_hook("feats"))
alexnet.classifier[1].register_forward_hook(reg_hook("feats"))


In [3]:
# transform data
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    ),
])


# path for the folder that contains the images
path = "../data/preprocess/"
cases = [f.path[19:] for f in os.scandir(path) if f.is_dir()]

# function to open images and return them as a list
def open_images(path: str, case: str, type: str) -> list:
    lst = []
    for filename in glob.glob(path + case + "/" + type + "/*.JPG"):
        lst.append(Image.open(filename).convert("RGB"))
    return lst

In [4]:
def extract_features(model: torchvision.models, transform: Callable, images: list) -> list:

    # function to normalize the vector
    def normalize(A: np.ndarray) -> np.ndarray:
        norm = np.linalg.norm(A)
        return A / norm

    embeddings = []
    
    # iterate through all images and save the features in the list
    for image in images:
        x = transform(image).unsqueeze(0).to("cpu")
        _ = model(x)

        embeddings.append(normalize(features["feats"].cpu().numpy()))
        
    return embeddings

In [5]:
def cosine_similarity(A: np.ndarray, B: np.ndarray) -> tuple:
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

def Average(lst):
    return sum(lst) / len(lst)

def distance_metric(embeddings: list) -> float:
    A = embeddings[0].flatten()
    B = embeddings[1].flatten()
    C = embeddings[2].flatten()

    c1 = cosine_similarity(A, B)
    c2 = cosine_similarity(A, C)

    def transform_dist_to_sim(x: float) -> float:
        return 1 / (1 + x)

    e1 = transform_dist_to_sim(distance.euclidean(A, B))
    e2 = transform_dist_to_sim(distance.euclidean(A, C))

    return (Average([c1, c2]), Average([e1, e2]))

### Using vgg16
Get the results using vgg16

In [6]:

for case in cases:
    
    images_before_treatment = open_images(path=path, case=case, type="BEFORE")
    images_after_treatment = open_images(path=path, case=case, type="AFTER")

    vgg16_before_embeddings = extract_features(model=vgg16, transform=transform, images=images_before_treatment)
    vgg16_after_embeddings = extract_features(model=vgg16, transform=transform, images=images_after_treatment)

    # get the distances
    before_vgg16_cos, before_vgg16_euc = distance_metric(vgg16_before_embeddings)
    after_vgg16_cos, after_vgg16_euc = distance_metric(vgg16_after_embeddings)

    # get the distance of their wounds only
    vgg16_wound_similarity = cosine_similarity(vgg16_before_embeddings[0].flatten(), vgg16_after_embeddings[0].flatten())

#     ## VGG19

#     vgg19_before_embeddings = extract_features(model=vgg19, transform=transform, images=images_before_treatment)
#     vgg19_after_embeddings = extract_features(model=vgg19, transform=transform, images=images_after_treatment)

#     # get the distances
#     before_vgg19_cos, before_vgg19_euc = distance_metric(vgg19_before_embeddings)
#     after_vgg19_cos, after_vgg19_euc = distance_metric(vgg19_after_embeddings)

#     # get the distance of their wounds only
#     vgg19_wound_similarity = cosine_similarity(vgg19_before_embeddings[0].flatten(), vgg19_after_embeddings[0].flatten())

#     ## ALEXNET
#     alexnet_before_embeddings = extract_features(model=alexnet, transform=transform, images=images_before_treatment)
#     alexnet_after_embeddings = extract_features(model=alexnet, transform=transform, images=images_after_treatment)

#     # get the distances
#     before_alexnet_cos, before_alexnet_euc = distance_metric(alexnet_before_embeddings)
#     after_alexnet_cos, after_alexnet_euc = distance_metric(alexnet_after_embeddings)

#     # get the distance of their wounds only
#     alexnet_wound_similarity = cosine_similarity(alexnet_before_embeddings[0].flatten(), alexnet_after_embeddings[0].flatten())

    data = []
    data.append(("vgg16", round(before_vgg16_cos,3), round(after_vgg16_cos, 3)))
#     data.append(("vgg16", round(before_vgg16_cos,3), round(after_vgg16_cos, 3), round(before_vgg16_euc, 3), round(after_vgg16_euc, 3), round(vgg16_wound_similarity, 3)))
#     data.append(("vgg19", round(before_vgg19_cos,3), round(after_vgg19_cos, 3), round(before_vgg19_euc, 3), round(after_vgg19_euc, 3), round(vgg19_wound_similarity, 3)))
#     data.append(("alexnet", round(before_alexnet_cos,3), round(after_alexnet_cos, 3), round(before_alexnet_euc, 3), round(after_alexnet_euc, 3), round(alexnet_wound_similarity, 3)))

#     df = pd.DataFrame(data, columns=['Net', 'cos sim (before)', 'cos sim (after)', 'euclidean (before)', 'euclidean (after)', 'cosine of the wound'])
    df = pd.DataFrame(data, columns=['Net', 'cos sim (before)', 'cos sim (after)'])
    df.to_csv(f'../csv/{case}/{case}_fully_connected.csv', index=False)


### Using vgg19

In [7]:
# vgg19_before_embeddings = extract_features(model=vgg19, transform=transform, images=images_before_treatment)
# vgg19_after_embeddings = extract_features(model=vgg19, transform=transform, images=images_after_treatment)

# # get the distances
# before_vgg19_cos, before_vgg19_euc = distance_metric(vgg19_before_embeddings)
# after_vgg19_cos, after_vgg19_euc = distance_metric(vgg19_after_embeddings)

# # get the distance of their wounds only
# vgg19_wound_similarity = cosine_similarity(vgg19_before_embeddings[0].flatten(), vgg19_after_embeddings[0].flatten())

### Using AlexNet

In [8]:
# alexnet_before_embeddings = extract_features(model=alexnet, transform=transform, images=images_before_treatment)
# alexnet_after_embeddings = extract_features(model=alexnet, transform=transform, images=images_after_treatment)

# # get the distances
# before_alexnet_cos, before_alexnet_euc = distance_metric(alexnet_before_embeddings)
# after_alexnet_cos, after_alexnet_euc = distance_metric(alexnet_after_embeddings)

# # get the distance of their wounds only
# alexnet_wound_similarity = cosine_similarity(alexnet_before_embeddings[0].flatten(), alexnet_after_embeddings[0].flatten())

In [9]:
# data = []
# data.append(("vgg16", round(before_vgg16_cos,3), round(after_vgg16_cos, 3), round(before_vgg16_euc, 3), round(after_vgg16_euc, 3), round(vgg16_wound_similarity, 3)))
# data.append(("vgg19", round(before_vgg19_cos,3), round(after_vgg19_cos, 3), round(before_vgg19_euc, 3), round(after_vgg19_euc, 3), round(vgg19_wound_similarity, 3)))
# data.append(("alexnet", round(before_alexnet_cos,3), round(after_alexnet_cos, 3), round(before_alexnet_euc, 3), round(after_alexnet_euc, 3), round(alexnet_wound_similarity, 3)))

# df = pd.DataFrame(data, columns=['Net', 'cos sim (before)', 'cos sim (after)', 'euclidean (before)', 'euclidean (after)', 'cosine of the wound'])
# df.to_csv(f'../csv/{case}/{case}_fully_connected.csv', index=False)